# Shared data directory (Experimental)

In this tutorial we will describe a shared directory `data`. Qiskit Functions can produce and consume files in the `data` directory.

`QiskitServerless` has `file_download`, `file_upload`, `file_delete` and `files` functions that provide file upload, file download, file delete and list files.

Limitations:

- files should be saved in `/data` directory during your function execution to be visible by `.files()` method call.
- only `/data` directory is supported, `/data/other_folder` will not be visible.
- as a provider you have access to `/function-data`, it works in a similar way as the `/data` folder with the distinction that users don't have access to it. Only the providers of the specific functions can see files under that path.
- Qiskit Functions created by you and Qiskit Functions created by others don't share directories.

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [2]:
import os
from qiskit_serverless import ServerlessClient, QiskitFunction

serverless = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost"),
    # If you are using the kubernetes approach the URL must be http://localhost
)
serverless

<gateway-client>

In [3]:
producer_function = QiskitFunction(
    title="file-producer", entrypoint="produce_files.py", working_dir="./source_files/"
)

serverless.upload(producer_function)

QiskitFunction(file-producer)

In [4]:
producer_function = serverless.get("file-producer")
producer_function

QiskitFunction(file-producer)

Upload a file in the `data` directory

In [5]:
import tarfile

filename = "uploaded_file.tar"
file = tarfile.open(filename, "w")
file.add("manage_data_directory.ipynb")
file.close()

In [6]:
serverless.file_upload(filename, producer_function)

'{"message":"/usr/src/app/media/mockuser/uploaded_file.tar"}'

Produce a file in the `data`directory

In [7]:
job = serverless.run("file-producer")
job

<Job | 18b9145a-45fc-401a-936a-de28619720de>

In [8]:
job.result()

{'Message': 'my_file.txt archived into my_file.tar'}

List files in the `data` directory

In [9]:
serverless.files(producer_function)

['uploaded_file.tar', 'my_file.tar']

Consume the file procuded by `file-producer` Qiskit Function

In [10]:
consumer_function = QiskitFunction(
    title="file-consumer", entrypoint="consume_files.py", working_dir="./source_files/"
)
serverless.upload(consumer_function)

QiskitFunction(file-consumer)

In [11]:
consumer_function = serverless.get("file-consumer")
consumer_function

QiskitFunction(file-consumer)

In [12]:
job = consumer_function.run()
job

<Job | 4d9dc660-97db-4e92-a578-42f2ea38ae0e>

In [13]:
job.result()

{'Message': "b'Hello!'"}

List files

In [14]:
serverless.files(consumer_function)

['uploaded_file.tar', 'my_file.tar']

Delete files

In [15]:
serverless.file_delete(filename, consumer_function)

'Requested file was deleted.'

In [16]:
serverless.files(consumer_function)

['my_file.tar']